<a href="https://colab.research.google.com/github/peterbrammer/sample_node_eb_paas/blob/master/faster_r_cnn_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.learnopencv.com/faster-r-cnn-object-detection-with-pytorch/